In [6]:
import pandas as pd
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
caminho_arquivo = '../data/0 - tabela de idades.xlsx'
df = pd.read_excel(caminho_arquivo, sheet_name="1) POP_IDADE SIMPLES", skiprows=5)

In [8]:
df = df[df['SEXO'] == 'Ambos'] # Filtra apenas o sexo "Ambos"

df = df[df['IDADE'] != 'Total'] # Remove linhas com idade "Total"

df['IDADE'] = df['IDADE'].astype(int) # Converte idade para inteiro

regioes_validas = ["Norte", "Nordeste", "Sudeste", "Sul", "Centro-Oeste"] # Filtra apenas as 5 grandes regiões

df = df[df['LOCAL'].isin(regioes_validas)] 

In [9]:
value_vars = [col for col in df.columns if isinstance(col, int) or (isinstance(col, str) and col.isnumeric())]

df_melt = df.melt(
    id_vars=['IDADE', 'SEXO', 'CÓD.', 'SIGLA', 'LOCAL'],
    value_vars=value_vars,
    var_name='ANO',
    value_name='POPULACAO'
)

df_melt['ANO'] = df_melt['ANO'].astype(int)
df_melt['POPULACAO'] = df_melt['POPULACAO'].astype(int)

In [ ]:
df_faixa = df_melt[(df_melt['IDADE'] >= 38) & (df_melt['IDADE'] <= 58)]

df_final = df_faixa.groupby(['LOCAL', 'ANO'], as_index=False)['POPULACAO'].sum()

df_final = df_final.rename(columns={'LOCAL': 'Regiao'})

df_final = df_final[(df_final['ANO'] >= 2007) & (df_final['ANO'] <= 2022)]

In [11]:
df_final.head(5)

,Regiao,ANO,POPULACAO
7,Centro-Oeste,2007,3159407
8,Centro-Oeste,2008,3265538
9,Centro-Oeste,2009,3371072
10,Centro-Oeste,2010,3474778
11,Centro-Oeste,2011,3576720


In [ ]:
# Criacao da pasta de 'data' caso não exista
parent_dir = os.path.dirname(os.getcwd())
output_dir = os.path.join(parent_dir, 'data')

os.makedirs(output_dir, exist_ok=True)

# Salvamento do arquivo
output_path = os.path.join(output_dir, '1 - populacao_igbe.csv')
df_final.to_csv(output_path, index=False)